# Schedule training

This script installs the dependencies in Colab, and schedules training of many notebooks. This notebook calls the training scripr "training.ipynb".

How to run:

1. Find the lines "TODO-USER" in this notebook to complete basic settings (e.g., set logging paths)
2. Find the lines "TODO-USER" in the notebook "training.ipynb" to set the paths
3. Simply run the script :-)

## Colab

In [0]:
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# @title Install Project
%cd /content
!git clone https://github.com/margiki/Interpretability-Adversarial.git
%cd /content/Interpretability-Adversarial
!pip install -r requirements.txt

# TODO-USER change these settings 

In [ ]:
# TODO-USER change these when running the notebook
# The data set is downloaded from kaggle.com. You need to create a free kaggle account to download the data, and then paste the username and key in here.
KAGGLE_USERNAME = None
KAGGLE_KEY = None
# Path for downloaded reproducibility material (e.g., data splits)
REPRODUCIBILITY_PATH = None (e.g., '/content/drive/My\ Drive/reproducibility')

In [ ]:
DATA_SPLIT_PATH = f"{REPRODUCIBILITY_PATH}/dataset_splits/*"
USER_MODELS_PATH = f"{REPRODUCIBILITY_PATH}/models"

In [0]:
!mkdir /root/.kaggle

import json
token = {"username":f"{KAGGLE_USERNAME}","key":f"{KAGGLE_KEY}"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!mkdir /content/data
!kaggle datasets download kmader/skin-cancer-mnist-ham10000 -p /content/data

In [0]:
# Unzip the whole zipfile into /content/data
!unzip -o /content/data/skin-cancer-mnist-ham10000.zip -d /content/data

!rm -rf /content/data/HAM10000_images_part_1
!rm -rf /content/data/HAM10000_images_part_2

%cd /content/data/ham10000_images_part_1/
!ls -1 | wc -l
%cd /content/data/ham10000_images_part_2/
!ls -1 | wc -l

In [ ]:
# Download the dataset splits
!gsutil -m cp -R $DATA_SPLIT_PATH /content/data/

In [0]:
%cd /content/Interpretability-Adversarial
!git pull

### Imports

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import papermill as pm
import json
import os
from datetime import datetime
from robustness.evaluation import plot_curves_from_file, plot_curves_from_log
from robustness import train, defaults, model_utils
from cox.utils import Parameters
from robustness.datasets import CIFAR, HAM10000_3cls, HAM10000_dataset_3cls_balanced
from robustness.tools.utils import fix_random_seed
from torch.utils.data.dataloader import DataLoader

## Training config

In [0]:
OUT_DIR = TODO-USER change this (e.g., "/content/drive/My Drive/notebooks_runs")
LOG_DIR = TODO-USER change this (e.g., "/content/drive/My Drive/logs")
DATA_DIR = TODO-USER change this (e.g., "/Users/andrei/Google Drive/data/HAM10000")

dataset_size = 2400
train_file_name = f'3cls_balanced_{dataset_size}_train_val.csv'
test_file_name = '3cls_balanced_test_without_val.csv'

In [0]:
def cross_validation_train(time_expid, val_fold, base_model_expid,
                           lr, train_file_name, test_file_name,
                           unfreeze_to_layer=-1,
                           BATCH_SIZE=16, step_lr=15, EPOCHS=25,
                           use_dropout_head=False, dropout_perc=0,
                           lr_patience=5, es_patience=10, custom_schedule='plateau', 
                           ADV_TRAIN=False, EPS=0, constraint='2', arch="resnet18",
                           apply_ablation=False, perc_ablation=0, 
                           do_eval_model=False, TRAIN_COLAB=True,
                           expid=None, seed=42):
  """
  Function to run the training for the cross_validation
  """
  saliency_dir = os.path.join(DATA_DIR, 'saliency_maps')
  if ADV_TRAIN:
    saliency_dir = os.path.join(saliency_dir, 'standard')
  else:
    saliency_dir = os.path.join(saliency_dir, f'adv {int(EPS)}')

  if expid == None:
    if params['ADV_TRAIN'] == False:
        params['expid'] = f"cv={val_fold}_full_std_lr={lr}_{dataset_size}_{time_expid}"
    else:
        params['expid'] = f"cv={val_fold}_full_adv_eps={EPS}_lr={lr}_{dataset_size}_{time_expid}"

  params = dict(
      TRAIN_COLAB = TRAIN_COLAB,
      ADV_TRAIN = ADV_TRAIN,
      lr = lr,
      BATCH_SIZE = BATCH_SIZE,
      EPOCHS = EPOCHS,
      base_model_expid = base_model_expid,
      unfreeze_to_layer = unfreeze_to_layer,
      arch = arch,
      step_lr = step_lr,
      custom_schedule = custom_schedule,
      train_file_name = f"{train_file_name}::{val_fold}",
      test_file_name = test_file_name,
      EPS = EPS,
      constraint = '2',
      apply_ablation = apply_ablation,
      saliency_dir = saliency_dir,
      perc_ablation=perc_ablation,
      do_eval_model=do_eval_model,
      lr_patience=lr_patience,
      es_patience=es_patience,
      eval_checkpoint_type='best',
      use_dropout_head = use_dropout_head,
      dropout_perc = dropout_perc,
      expid = expid,
      seed=seed
  )

  print(params['expid'])

  pm.execute_notebook(
        './notebooks/training.ipynb',
        os.path.join(OUT_DIR, f"{params['expid']}.ipynb"),
        parameters = params,
        nest_asyncio=True
    )

# Transfer learning experiment

In [0]:
# Transfer learning experiment - Unfreeze until a given layer
time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
print(time_expid)

for eps in [3, 4, 7]:
  for unfreeze_to_layer in [5]:
    cross_validation_train(time_expid, val_fold=0, base_model_expid=f'full_stability_eps={eps}_seed=1_2020-06-07---11:18:40',
                            train_file_name=f'3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                            step_lr=None, arch='resnet18', do_eval_model=True,
                            unfreeze_to_layer=unfreeze_to_layer, use_dropout_head=False,
                            lr=1e-5, custom_schedule='plateau', lr_patience=5, es_patience=10,
                            EPOCHS=50, BATCH_SIZE=16,
                            ADV_TRAIN=False,
                            expid=f'transfer_learning_unfreezeto={unfreeze_to_layer}_eps={eps}_{time_expid}',
                            seed=1)

## Finding the right learning rate for the Dropout models

In [0]:
# Dropout HEAD
for lr in [1e-3, 3e-4, 1e-4, 3e-5]:
  for dropout in [0]:
    time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
    print(time_expid)
    cross_validation_train(time_expid, val_fold=0, base_model_expid=None,
                          train_file_name=f'3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                          step_lr=15, arch='resnet18', do_eval_model=False,
                          use_dropout_head=True, dropout_perc=dropout,
                          lr=lr, custom_schedule=None, lr_patience=5, es_patience=10,
                          EPOCHS=15, BATCH_SIZE=32,
                          ADV_TRAIN=False,
                          expid = f"head_dropout_{dropout}_lr={lr}")

In [0]:
# Dropout BODY
# Fine-tuning for varying Nevi size

for lr in [3e-4, 1e-4, 3e-5, 1e-5]:
  for dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
    print(time_expid)
    cross_validation_train(time_expid, val_fold=0, base_model_expid=f"head_dropout_0_lr=0.001",
                            train_file_name=f'3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                            step_lr=None, arch='resnet18', do_eval_model=True,
                            use_dropout_head=True, dropout_perc=dropout,
                            lr=lr, custom_schedule='plateau', lr_patience=5, es_patience=10,
                            EPOCHS=50, BATCH_SIZE=16,
                            ADV_TRAIN=False,
                            expid=f'full_dropout_{dropout}_adv={0}_lr={lr}')

## Stability Experiment

In [0]:
time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
print(time_expid)

for seed in [1, 2, 3, 4, 5]:
  # HEAD
  cross_validation_train(time_expid, val_fold=0, base_model_expid=None,
                          train_file_name='3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                          step_lr=15, arch='resnet18', do_eval_model=False,
                          use_dropout_head=False, dropout_perc=0,
                          lr=1e-3, custom_schedule=None, lr_patience=5, es_patience=10,
                          EPOCHS=10, BATCH_SIZE=32,
                          ADV_TRAIN=False,
                          expid = f"head_stability_seed={seed}_{time_expid}", seed=seed)
  
  # FINE-TUNE
  cross_validation_train(time_expid, val_fold=0, base_model_expid=f"head_stability_seed={seed}_{time_expid}",
                          train_file_name='3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                          step_lr=None, arch='resnet18', do_eval_model=True,
                          lr=3e-4, custom_schedule='plateau', lr_patience=5, es_patience=10,
                          EPOCHS=50, BATCH_SIZE=16,
                          ADV_TRAIN=False,
                          expid=f'full_stability_seed={seed}_{time_expid}', seed=seed)

In [0]:
time_expid = '2020-06-07---11:18:40'
# ADV_TRAIN
for EPS in [7]:
  for seed in [1, 2, 3, 4, 5]:
    # FINE-TUNE
    cross_validation_train(time_expid, val_fold=0, base_model_expid=f"head_stability_seed={seed}_{time_expid}",
                            train_file_name='3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                            step_lr=None, arch='resnet18', do_eval_model=True,
                            lr=3e-4, custom_schedule='plateau', lr_patience=5, es_patience=10,
                            EPOCHS=50, BATCH_SIZE=16,
                            ADV_TRAIN=True, EPS=EPS,
                            expid=f'full_stability_eps={EPS}_seed={seed}_{time_expid}', seed=seed)

## Train Head

In [0]:
# Train head for Varying Training size ration
for nevi_cnt in [50, 100, 200, 400, 800, 1600, 3200]:
  time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
  print(time_expid)
  cross_validation_train(time_expid, val_fold=0, base_model_expid=None,
                        train_file_name=f'varying_training_ratio/3cls_balanced_{nevi_cnt}nevi_train_val.csv', test_file_name=test_file_name,
                        step_lr=15, arch='resnet18', do_eval_model=False,
                        use_dropout_head=False, dropout_perc=0,
                        lr=1e-3, custom_schedule=None, lr_patience=5, es_patience=10,
                        EPOCHS=10, BATCH_SIZE=32,
                        ADV_TRAIN=False,
                        expid = f"head_nevi_{nevi_cnt}")

In [0]:
# Dropout
for dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
  time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
  print(time_expid)
  cross_validation_train(time_expid, val_fold=0, base_model_expid=None,
                        train_file_name=f'3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                        step_lr=15, arch='resnet18', do_eval_model=False,
                        use_dropout_head=True, dropout_perc=dropout,
                        lr=1e-3, custom_schedule=None, lr_patience=5, es_patience=10,
                        EPOCHS=10, BATCH_SIZE=32,
                        ADV_TRAIN=False,
                        expid = f"head_dropout_{dropout}")

## Training only one fold

In [0]:
# Fine-tuning for varying Nevi size
for EPS in [3, 4, 5]:
  adv_train = EPS != 0
  print(adv_train)

  for nevi_cnt in [50, 100, 200, 400, 800, 1600, 3200]:
    time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
    print(time_expid)
    cross_validation_train(time_expid, val_fold=0, base_model_expid=f"head_nevi_{nevi_cnt}",
                           train_file_name=f'varying_training_ratio/3cls_balanced_{nevi_cnt}nevi_train_val.csv', test_file_name=test_file_name,
                           step_lr=None, arch='resnet18', do_eval_model=True,
                           lr=3e-4, custom_schedule='plateau', lr_patience=5, es_patience=10,
                           EPOCHS=50, BATCH_SIZE=16,
                           ADV_TRAIN=adv_train, EPS=EPS,
                           expid=f'full_nevi_{nevi_cnt}_adv={EPS}')

In [0]:
# Fine-tuning for varying Nevi size
for EPS in [3, 4, 5]:
  adv_train = EPS != 0
  print(adv_train)

  for dropout in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
    print(time_expid)
    cross_validation_train(time_expid, val_fold=0, base_model_expid=f"head_dropout_{dropout}",
                           train_file_name=f'3cls_balanced_2400_train_val.csv', test_file_name=test_file_name,
                           step_lr=None, arch='resnet18', do_eval_model=True,
                           use_dropout_head=True, dropout_perc=dropout,
                           lr=3e-4, custom_schedule='plateau', lr_patience=5, es_patience=10,
                           EPOCHS=50, BATCH_SIZE=16,
                           ADV_TRAIN=adv_train, EPS=EPS,
                           expid=f'full_dropout_{dropout}_adv={EPS}')

## Full-body cross-validation

In [0]:
for lr in [1e-4, 1e-3]:
  for batch_size in [16]:
    time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
    print(time_expid)
    for val_fold, base_model_expid in zip([1, 2, 3, 4, 5], ['cv=1_full_std_lr=0.001_2400_2020-05-24---05:38:16', 
                                                             'cv=2_full_std_lr=0.001_2400_2020-05-24---05:38:16',
                                                             'cv=3_full_std_lr=0.001_2400_2020-05-24---05:38:16',
                                                             'cv=4_full_std_lr=0.001_2400_2020-05-24---05:38:16',
                                                             'cv=5_full_std_lr=0.001_2400_2020-05-24---05:38:16']):

        cross_validation_train(time_expid, val_fold=val_fold, base_model_expid=base_model_expid, file_name=file_name,
                              lr=lr, BATCH_SIZE=batch_size,
                              step_lr=None, EPOCHS=30, arch='resnet50',
                              ADV_TRAIN=False)

# Train on the entire dataset

In [0]:
time_expid = datetime.now().strftime('%Y-%m-%d---%H:%M:%S')
print(time_expid)

for EPS in [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9]:
  cross_validation_train(time_expid, val_fold=0, base_model_expid='cv=1_full_std_lr=0.001_2400_2020-05-24---05:38:16', file_name=file_name,
                          lr=3e-4, BATCH_SIZE=32,
                          step_lr=None, EPOCHS=30, arch='resnet50',
                          ADV_TRAIN=True, EPS=EPS, constraint='2')

# Evaluation

In [0]:
EPS = 3
attack_kwargs = {
    'constraint': '2',
    'eps': EPS,
    'attack_lr': 2.5 * EPS / 30,
    'attack_steps': 30,
    'random_start': True
}
adv_eval_args = Parameters({**attack_kwargs, **{'adv_eval': False}})
adv_eval_args = defaults.check_and_fill_args(adv_eval_args, defaults.PGD_ARGS, CIFAR)
adv_eval_args

In [0]:
dataset = HAM10000_3cls("/content/data", file_name="3cls_balanced_test.csv")

In [0]:
# ----- TEST loader
test_dataset = HAM10000_dataset_3cls_balanced(dataset.data_path, "3cls_balanced_test.csv", transform=dataset.transform_test, test=True) 
loader = DataLoader(test_dataset, batch_size=16, num_workers=16, pin_memory=True)

In [0]:
fix_random_seed(42)

model, _ = model_utils.make_and_restore_model(
    arch='resnet50',
    dataset=dataset,
    resume_path=os.path.join(LOG_DIR, 'cv=0_full_adv_eps=3_lr=0.0003_2400_2020-05-13---08:57:50', 'checkpoint.pt.latest'),
    device='cuda')

In [0]:
train.eval_model(adv_eval_args, model, loader, None)

# Evaluate Cross_validation
Get all folds from cross-validation and compute mean and std of the accuracy.

In [0]:
def read_log_cross_validation(logs_dir, exp_id):
  train_accs, val_accs = [], []
  for fold in [1, 2, 3, 4, 5]:
    log = cox.store.Store(logs_dir, f"cv={fold}_{exp_id}")
    last_row = log.tables['logs'].df.iloc[-1]
    log.close()

    train_accs.append(last_row.train_prec1)
    val_accs.append(last_row.nat_prec1)

  print("Train:  %.2f +- %.2f" % (np.mean(train_accs), np.std(train_accs)))
  print("Valid:  %.2f +- %.2f" % (np.mean(val_accs), np.std(val_accs)))

read_log_cross_validation(LOG_DIR, 'full_std_lr=0.001_2400_2020-05-24---17:53:45')